# Modelos "feijão com arroz" utilizando TF-IDF para vetorização

### Downloads

In [ ]:
%pip install scikit-learn tabulate

### Imports

In [2]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from tabulate import tabulate

### Treinando um modelo Naive Bayes
Aqui foi adicionado o parâmetro "sublinear_tf=True" no TfidfVectorizer que o deixa mais sensível a termos menos frequentes mas potencialmente mais significativos, que, no geral, melhorou sua acurácia nos datasets.

In [3]:
def train_model(csv_path):
  df_train = pd.read_csv("CSV/" + csv_path)
  df_train = df_train.fillna('')

  if "alternativas_separadas" in csv_path:
    # Vetorização do texto usando TF-IDF
    vectorizer = TfidfVectorizer(sublinear_tf=True)
    X_train_text = vectorizer.fit_transform(df_train['Enunciado'])

    # Vetoriza as colunas de alternativa
    alternative_columns = ['Alternativa_A', 'Alternativa_B', 'Alternativa_C', 'Alternativa_D', 'Alternativa_E']
    for col in alternative_columns:
        X_train_text += vectorizer.transform(df_train[col])

  else:
    # Vetorização do texto usando TF-IDF
    vectorizer = TfidfVectorizer(sublinear_tf=True)
    X_train_text = vectorizer.fit_transform(df_train['Enunciado_Alternativas'])

  # Divide os dados em conjunto de treinamento e teste
  X_train, X_test, y_train, y_test = train_test_split(X_train_text, df_train['Area_de_Conhecimento'], test_size=0.2, random_state=3)

  # Treinamento do modelo
  model = MultinomialNB()
  model.fit(X_train, y_train)

  # Predição no conjunto de teste
  y_pred_train = model.predict(X_test)

  # Avaliação do modelo
  print(f"Avaliação do modelo no conjunto {csv_path}:\n")
  print(classification_report(y_test, y_pred_train, target_names=['Matemática', 'Linguagens', 'Ciências Humanas', 'Ciências da Natureza']))
  print("-"*62)

  return classification_report(y_test, y_pred_train, output_dict=True)['accuracy']

accuracies_naive_bayes = []
for filename in os.listdir('CSV'):
   accuracies_naive_bayes.append(train_model(filename))

Avaliação do modelo no conjunto alternativas_separadas.csv:

                      precision    recall  f1-score   support

          Matemática       0.95      0.66      0.78        32
          Linguagens       0.88      0.70      0.78        30
    Ciências Humanas       0.78      1.00      0.88        47
Ciências da Natureza       0.87      0.93      0.90        42

            accuracy                           0.85       151
           macro avg       0.87      0.82      0.83       151
        weighted avg       0.86      0.85      0.84       151

--------------------------------------------------------------
Avaliação do modelo no conjunto alternativas_separadas_pp.csv:

                      precision    recall  f1-score   support

          Matemática       0.89      0.75      0.81        32
          Linguagens       0.86      0.80      0.83        30
    Ciências Humanas       0.87      1.00      0.93        47
Ciências da Natureza       0.88      0.88      0.88        42

 

### Treinando um modelo Logistic Regression
Aqui foi adicionado o parâmetro "sublinear_tf=True" no TfidfVectorizer que o deixa mais sensível a termos menos frequentes mas potencialmente mais significativos, que, no geral, melhorou sua acurácia nos datasets.

In [4]:
def train_model(csv_path):
  df_train = pd.read_csv("CSV/" + csv_path)
  df_train = df_train.fillna('')

  if "alternativas_separadas" in csv_path:
    # Vetorização do texto usando TF-IDF
    vectorizer = TfidfVectorizer(sublinear_tf=True)
    X_train_text = vectorizer.fit_transform(df_train['Enunciado'])

    # Vetoriza as colunas de alternativa
    alternative_columns = ['Alternativa_A', 'Alternativa_B', 'Alternativa_C', 'Alternativa_D', 'Alternativa_E']
    for col in alternative_columns:
        X_train_text += vectorizer.transform(df_train[col])

  else:
    # Vetorização do texto usando TF-IDF
    vectorizer = TfidfVectorizer(sublinear_tf=True)
    X_train_text = vectorizer.fit_transform(df_train['Enunciado_Alternativas'])

  # Divide os dados em conjunto de treinamento e teste
  X_train, X_test, y_train, y_test = train_test_split(X_train_text, df_train['Area_de_Conhecimento'], test_size=0.2, random_state=3)

  # Treinamento do modelo
  model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
  model.fit(X_train, y_train)

  # Predição no conjunto de teste
  y_pred_train = model.predict(X_test)

  # Avaliação do modelo
  print(f"Avaliação do modelo no conjunto {csv_path}:\n")
  print(classification_report(y_test, y_pred_train, target_names=['Matemática', 'Linguagens', 'Ciências Humanas', 'Ciências da Natureza']))
  print("-"*62)

  return classification_report(y_test, y_pred_train, output_dict=True)['accuracy']

accuracies_logistic_regression = []
for filename in os.listdir('CSV'):
   accuracies_logistic_regression.append(train_model(filename))

Avaliação do modelo no conjunto alternativas_separadas.csv:

                      precision    recall  f1-score   support

          Matemática       0.89      0.78      0.83        32
          Linguagens       0.76      0.83      0.79        30
    Ciências Humanas       0.90      0.91      0.91        47
Ciências da Natureza       0.88      0.88      0.88        42

            accuracy                           0.86       151
           macro avg       0.86      0.85      0.85       151
        weighted avg       0.86      0.86      0.86       151

--------------------------------------------------------------
Avaliação do modelo no conjunto alternativas_separadas_pp.csv:

                      precision    recall  f1-score   support

          Matemática       0.81      0.81      0.81        32
          Linguagens       0.87      0.90      0.89        30
    Ciências Humanas       0.94      0.94      0.94        47
Ciências da Natureza       0.93      0.90      0.92        42

 

### Treinando um modelo Random Forest

In [5]:
def train_model(csv_path):
  df_train = pd.read_csv("CSV/" + csv_path)
  df_train = df_train.fillna('')

  if "alternativas_separadas" in csv_path:
    # Vetorização do texto usando TF-IDF
    vectorizer = TfidfVectorizer()
    X_train_text = vectorizer.fit_transform(df_train['Enunciado'])

    # Vetoriza as colunas de alternativa
    alternative_columns = ['Alternativa_A', 'Alternativa_B', 'Alternativa_C', 'Alternativa_D', 'Alternativa_E']
    for col in alternative_columns:
        X_train_text += vectorizer.transform(df_train[col])

  else:
    # Vetorização do texto usando TF-IDF
    vectorizer = TfidfVectorizer()
    X_train_text = vectorizer.fit_transform(df_train['Enunciado_Alternativas'])

  # Divide os dados em conjunto de treinamento e teste
  X_train, X_test, y_train, y_test = train_test_split(X_train_text, df_train['Area_de_Conhecimento'], test_size=0.2, random_state=3)

  # Treinamento do modelo
  model = RandomForestClassifier(n_estimators=200, random_state=3)
  model.fit(X_train, y_train)

  # Predição no conjunto de teste
  y_pred_train = model.predict(X_test)

  # Avaliação do modelo
  print(f"Avaliação do modelo no conjunto {csv_path}:\n")
  print(classification_report(y_test, y_pred_train, target_names=['Matemática', 'Linguagens', 'Ciências Humanas', 'Ciências da Natureza']))
  print("-"*62)

  return classification_report(y_test, y_pred_train, output_dict=True)['accuracy']

accuracies_random_forest = []
for filename in os.listdir('CSV'):
   accuracies_random_forest.append(train_model(filename))

Avaliação do modelo no conjunto alternativas_separadas.csv:

                      precision    recall  f1-score   support

          Matemática       0.81      0.53      0.64        32
          Linguagens       0.69      0.83      0.76        30
    Ciências Humanas       0.76      0.89      0.82        47
Ciências da Natureza       0.87      0.81      0.84        42

            accuracy                           0.78       151
           macro avg       0.78      0.77      0.77       151
        weighted avg       0.79      0.78      0.78       151

--------------------------------------------------------------
Avaliação do modelo no conjunto alternativas_separadas_pp.csv:

                      precision    recall  f1-score   support

          Matemática       0.79      0.69      0.73        32
          Linguagens       0.84      0.90      0.87        30
    Ciências Humanas       0.86      0.89      0.88        47
Ciências da Natureza       0.93      0.93      0.93        42

 

### Comparando acurácias

In [7]:
datasets = ['alternativas_separadas', 'alternativas_separadas_pp', 'tudo_junto', 'tudo_junto_pp']

df_accuracies = pd.DataFrame({
    'Dataset': datasets,
    'Naive Bayes': accuracies_naive_bayes,
    'Logistic Regression': accuracies_logistic_regression,
    'Random Forest': accuracies_random_forest
})

df_accuracies['Média de Acurácia'] = df_accuracies.mean(axis=1, numeric_only=True)

print(tabulate(df_accuracies, headers='keys', tablefmt='fancy_grid', showindex=False))

╒═══════════════════════════╤═══════════════╤═══════════════════════╤═════════════════╤═════════════════════╕
│ Dataset                   │   Naive Bayes │   Logistic Regression │   Random Forest │   Média de Acurácia │
╞═══════════════════════════╪═══════════════╪═══════════════════════╪═════════════════╪═════════════════════╡
│ alternativas_separadas    │      0.847682 │              0.860927 │        0.781457 │            0.830022 │
├───────────────────────────┼───────────────┼───────────────────────┼─────────────────┼─────────────────────┤
│ alternativas_separadas_pp │      0.874172 │              0.89404  │        0.860927 │            0.87638  │
├───────────────────────────┼───────────────┼───────────────────────┼─────────────────┼─────────────────────┤
│ tudo_junto                │      0.854305 │              0.854305 │        0.761589 │            0.8234   │
├───────────────────────────┼───────────────┼───────────────────────┼─────────────────┼─────────────────────┤
│ tudo_jun